In [1]:
import sys
import os

PROJECT_ROOT = "/Users/kevingarrison/Code Projects/Private Projects/Agents/Langchain_LECL/langchain-lecl"
SRC_PATH = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_docling.loader import ExportType
from langchain_openai import OpenAIEmbeddings
from docling.chunking import HybridChunker
from langchain.agents import create_agent
from langchain.tools import tool
from dotenv import load_dotenv
from typing import Literal


load_dotenv()

FILE_PATH = ["/Users/kevingarrison/Code Projects/Private Projects/Agents/Langchain_LECL/langchain-lecl/notebooks/Bachelorarbeit_Kevin_Garrison_85826.pdf"]
EMBED_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
EXPORT_TYPE = ExportType.DOC_CHUNKS
QUESTION = "Which are the main AI models in Docling?"
PROMPT = PromptTemplate.from_template(
    "Context information is below.\n---------------------\n{context}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {input}\nAnswer:\n",
)
TOP_K = 3

In [3]:
from langchain_docling import DoclingLoader

from docling.chunking import HybridChunker

loader = DoclingLoader(
    file_path=FILE_PATH,
    export_type=EXPORT_TYPE,
    chunker=HybridChunker(tokenizer=EMBED_MODEL_ID),
)

docs = loader.load()

2025-11-24 12:40:24,947 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-11-24 12:40:24,988 - INFO - Going to convert document batch...
2025-11-24 12:40:24,988 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 44ae89a68fc272bc7889292e9b5a1bad
2025-11-24 12:40:24,993 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-11-24 12:40:24,994 - INFO - Loading plugin 'docling_defaults'
2025-11-24 12:40:24,995 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-11-24 12:40:24,999 - WARNING - The plugin langchain_docling will not be loaded because Docling is being executed with allow_external_plugins=false.
2025-11-24 12:40:24,999 - INFO - Loading plugin 'docling_defaults'
2025-11-24 12:40:25,002 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-11-24 12:40:25,414 - INFO - Auto OCR model selected ocrmac.
2025-11-24 12

In [5]:
if EXPORT_TYPE == ExportType.DOC_CHUNKS:
    splits = docs
elif EXPORT_TYPE == ExportType.MARKDOWN:
    from langchain_text_splitters import MarkdownHeaderTextSplitter

    splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header_1"),
            ("##", "Header_2"),
            ("###", "Header_3"),
        ],
    )
    splits = [split for doc in docs for split in splitter.split_text(doc.page_content)]
else:
    raise ValueError(f"Unexpected export type: {EXPORT_TYPE}")

In [6]:
for d in splits[:3]:
    print(f"- {d.page_content=}")
print("...")

- d.page_content='STUTTGART\nPORSCHe\nBachelorarbeit Studiengang : Data Science'
- d.page_content='Einsatz von Large Language Models (LLM) zur Extraktion und Strukturierung von Zolldokumenten: Ein KI-gestützter Ansatz zur automatisierten Datenverarbeitung\nbei Porsche AG von Kevin Garrison\n85826\nBetreuender Professor: Prof. Dr. Winfried Bantel Zweitprüfer : Prof. Dr. Tim Dahmen\nEinreichungsdatum : 14. August 2025'
- d.page_content='Angaben zur Firma\nUnternehmen :\nPorsche AG\nBranche :\nAutomobilbranche Finanzstrategie & Data Science Porscheplatz 1 D - 70435 Stuttgart\nAbteilung :\nAdresse :\nBetreuerin :\nMaike Klepsch (FOD) (+49) 0 152 3 911 0075 maike.klepsch@porsche.de\nTelefon :\nE-Mail :'
...


In [7]:
print(f"Split blog post into {len(splits)} sub-documents.")

Split blog post into 327 sub-documents.


In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


vector_store = InMemoryVectorStore(embeddings)


document_ids = vector_store.add_documents(documents=splits)

print(document_ids[:3])

2025-11-24 12:40:46,671 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


['eb73401a-b3e3-45fd-b15e-512a157a923e', 'fd7d7638-5a8d-4efb-80bf-81dea9cf6566', '9f16d1eb-9e48-4c27-b0bb-f76a04f8ef81']


In [9]:
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=TOP_K)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [10]:
tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)

model = init_chat_model("gpt-4.1")

agent = create_agent(model, tools, system_prompt=prompt)

In [11]:
query = (
    "What are the key findings of the paper?\n\n"
    "Once you get the answer, retrive the ."
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What are the key findings of the paper?

Once you get the answer, retrive the .


2025-11-24 12:40:49,168 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================
Tool Calls:
  retrieve_context (call_d8nqZBzwrZ3aZgUFlBwR3Pbk)
 Call ID: call_d8nqZBzwrZ3aZgUFlBwR3Pbk
  Args:
    query: key findings of the paper


2025-11-24 12:40:49,459 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


================================= Tool Message =================================
Name: retrieve_context

Source: {'source': '/Users/kevingarrison/Code Projects/Private Projects/Agents/Langchain_LECL/langchain-lecl/notebooks/Bachelorarbeit_Kevin_Garrison_85826.pdf', 'dl_meta': {'schema_name': 'docling_core.transforms.chunker.DocMeta', 'version': '1.0.0', 'doc_items': [{'self_ref': '#/texts/27', 'parent': {'$ref': '#/body'}, 'children': [], 'content_layer': 'body', 'label': 'text', 'prov': [{'page_no': 5, 'bbox': {'l': 88.931, 't': 642.6360146484375, 'r': 507.798, 'b': 511.0610146484375, 'coord_origin': 'BOTTOMLEFT'}, 'charspan': [0, 750]}]}], 'headings': ['Kurzfassung'], 'origin': {'mimetype': 'application/pdf', 'binary_hash': 18027861618462669308, 'filename': 'Bachelorarbeit_Kevin_Garrison_85826.pdf'}}}
Content: Kurzfassung
Ergebnisse hinzunehmen.

Source: {'source': '/Users/kevingarrison/Code Projects/Private Projects/Agents/Langchain_LECL/langchain-lecl/notebooks/Bachelorarbeit_Kevin

2025-11-24 12:40:53,360 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


================================== Ai Message ==================================

Here are the key findings of the paper:

- The classic system's performance is significantly impaired by erroneous input data. In contrast, the LLM-based (Large Language Model) approach can partially compensate for these weaknesses through contextual interpretation.
- The best results with the LLM approach were achieved with a recall of 0.53 and an F1-score of 0.60, especially when there was exact numeric value matching combined with a Levenshtein threshold above 90% for string comparisons to test data.
- The results indicate that the LLM approach is more robust to faulty input data and demonstrates a higher degree of generalization capability. However, it also has its own limitations regarding reproducibility and extraction precision.
- The system's performance heavily depends on the quality and structure of the underlying data.
- The empirical analysis reveals that the current extraction pipeline is sti